Exploring stuff for segmentation

In [ ]:
%load_ext autoreload
%autoreload 2

import pathlib

parent_dir = pathlib.Path("~/zebrafish_rdsf/Rabia/SOST scales").expanduser()
assert parent_dir.exists()

scale_dirs = tuple(d for d in parent_dir.glob("*") if not d.stem in {".DS_Store", "TIFs"})

In [ ]:
from tqdm.notebook import tqdm

from scale_morphology.scales import read

# Just look at the ALP scales since they seem easiest to segment
(alp_dir,) = (d for d in scale_dirs if "ALP" in d.stem)

alp_files = list(alp_dir.glob("*.lif"))
print(len(alp_files), "files")

names, images = [], []
for path in alp_files:
    name, img = zip(*read.read_lif(path))
    names += name
    images += img

In [ ]:
out_dir = pathlib.Path("segmentation_stuff")

overlaid_dir = out_dir / "overlaid"
mask_dir = out_dir / "masks"
img_dir = out_dir / "imgs"

for d in overlaid_dir, mask_dir, img_dir:
    d.mkdir(exist_ok=True, parents=True)

In [ ]:
"""
Build our training set by first using classical CV to perform segmentation
"""

import tifffile
from matplotlib import colors
import matplotlib.pyplot as plt
from scale_morphology.scales import segmentation


def clear2black_cmap() -> colors.Colormap:
    """
    Colormap that varies from clear to black
    """
    c_white = colors.colorConverter.to_rgba("white", alpha=0)
    c_black = colors.colorConverter.to_rgba("black", alpha=1)
    return colors.ListedColormap([c_white, c_black], "clear2black")


cmap = clear2black_cmap()
for name, img in zip(tqdm(names), images):
    mask_path = mask_dir / (name + ".tif")
    if mask_path.exists():
        continue
    mask = segmentation.classical_segmentation(img)

    tifffile.imwrite(mask_path, mask)
    tifffile.imwrite(img_dir / (name + ".tif"), img)

    fig, axis = plt.subplots(figsize=(5, 5))
    axis.imshow(img)
    axis.imshow(mask, cmap=cmap, alpha=0.5)
    axis.set_axis_off()
    fig.savefig(overlaid_dir / (name + ".png"))
    plt.close(fig)

In [ ]:
"""
Tidy the masks in napari
"""

import napari
import numpy as np
import tifffile
from tqdm.notebook import tqdm

cleaned_dir = out_dir / "cleaned_masks"
cleaned_dir.mkdir(exist_ok=True)

mask_paths = list(mask_dir.glob("*.tif"))
for mask_path in tqdm(mask_paths):
    name = mask_path.name

    img_path = img_dir / name
    out_path = cleaned_dir / name

    im = tifffile.imread(img_path)
    mask = tifffile.imread(mask_path).astype(np.uint8)

    viewer = napari.Viewer()
    viewer.add_image(im, name=f"image:{name}")

    labels = viewer.add_labels(mask, name=f"mask:{name}", opacity=0.5)
    labels.selected_label = 1

    state = {"saved": False, "skip": False}

    @viewer.bind_key("s")
    def _save(_):
        tifffile.imwrite(out_path, (labels.data > 0).astype(np.uint8) * 255)
        state["saved"] = True
        print(f"saved {out_path}")

    @viewer.bind_key("x")
    def _skip(_):
        state["skip"] = True
        print(f"Skip {out_path}")

    napari.run()
    if not state["skip"] and not state["saved"]:
        tifffile.imwrite(out_path, (labels.data > 0).astype(np.uint8) * 255)
        print(f"Autosaved {out_path}")
    else:
        print("skipped saving")